In [1]:
import openai
import os
import pandas as pd
import time

movie_df = pd.read_csv('../Resources/sentiment_movies.csv')
movie_df = movie_df.drop(columns=['Unnamed: 0']) 
movie_df.head()

,movie_id,popularity,title,vote_average,vote_count,cleaned_reviews,type_Action,type_Adventure,type_Animation,type_Comedy,...,review_neu,review_pos,review_compound,clean_overview,overview_neg,overview_neu,overview_pos,overview_compound,release_date,release_year
0,693134,4534.956,Dune: Part Two,8.311,2707,dune part two surpasses even highest expectati...,False,True,False,False,...,0.646,0.286,0.9995,follow mythic journey paul atreides unites cha...,0.269,0.583,0.147,-0.6597,2024-02-27,2024
1,1011985,3214.314,Kung Fu Panda 4,7.066,897,kung fu panda best kung fu panda film even bes...,True,True,True,True,...,0.542,0.205,-0.9793,po gearing become spiritual leader valley peac...,0.390,0.526,0.084,-0.9300,2024-03-02,2024
2,823464,2297.340,Godzilla x Kong: The New Empire,6.700,575,godzilla kong new empire version previous inst...,True,True,False,False,...,0.716,0.173,0.9799,following explosive showdown godzilla kong mus...,0.189,0.722,0.089,-0.4215,2024-03-27,2024
3,359410,1134.726,Road House,7.066,1379,poor scenario story add ending also convincing...,True,False,False,False,...,0.553,0.336,0.9616,exufc fighter dalton takes job bouncer florida...,0.000,0.633,0.367,0.7003,2024-03-08,2024
4,634492,921.140,Madame Web,5.676,1046,start heavily pregnant woman deep peruvian jun...,True,False,False,False,...,0.679,0.217,0.9949,forced confront revelations past paramedic cas...,0.209,0.667,0.124,-0.2263,2024-02-14,2024


In [2]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6561 entries, 0 to 6560
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_id              6561 non-null   int64  
 1   popularity            6561 non-null   float64
 2   title                 6561 non-null   object 
 3   vote_average          6561 non-null   float64
 4   vote_count            6561 non-null   int64  
 5   cleaned_reviews       6561 non-null   object 
 6   type_Action           6561 non-null   bool   
 7   type_Adventure        6561 non-null   bool   
 8   type_Animation        6561 non-null   bool   
 9   type_Comedy           6561 non-null   bool   
 10  type_Crime            6561 non-null   bool   
 11  type_Documentary      6561 non-null   bool   
 12  type_Drama            6561 non-null   bool   
 13  type_Family           6561 non-null   bool   
 14  type_Fantasy          6561 non-null   bool   
 15  type_History         

# The base template for a successful use of openai

In [17]:
# # load_dotenv()
# # import warnings
# # from transformers import pipeline
# openai.api_key = os.getenv("OPENAI_API_KEY")

# def get_sentiment(question):
#     messages = [
#         {"role": "system","content": "Determine if the provided text is happy, sad, or neutral"},
#         {"role": "user", "content": question}
#     ]
    
#     response = openai.chat.completions.create(
#         model ="gpt-3.5-turbo",
#         messages=messages
#     )
    
#     answer = response.choices[0].message.content
#     return answer
# # # example q
# # # question = "The sun is crying rainbows"
# # answer = get_sentiment(text)
# # print(text)
# # print(answer)

# movie_df = pd.read_csv('../Resources/sentiment_movies.csv')  # Adjust the path as necessary

# # Apply sentiment analysis to each review in the cleaned_reviews column
# movie_df['sentiment'] = movie_df['cleaned_reviews'].apply(get_sentiment)

# # Print a few results to verify
# print(movie_df[['cleaned_reviews', 'sentiment']].head())

# # Optionally, save the results back to a CSV
# movie_df.to_csv('openai_sentiment_of_reviews.csv', index=False)

# Possible routes to take:

# Below:
- The system responds with a detailed answer on the emotional state a person may be in after watching said movie, based on it's reviews. We could pull these words out?
#### The prompt:
- "Based on the text provided, please perform a sentiment analysis. Given that these are movie reviews, please determine which emotional state a person may be in if they were to watch a movie given the reviews. Explain why."
---
- An example from the Dune 2 movie:
    - "Overall, the sentiment in the reviews seems to be overwhelmingly positive. Viewers would likely feel excited, intrigued, and engaged while watching a movie based on these reviews. There are numerous mentions of the technical brilliance, immersive experience, breathtaking visuals, outstanding performances, and the elevation of the cinematic experience to unparalleled heights. These elements would evoke emotions such as excitement, awe, and inspiration in the audience. Additionally, the complex narrative, deep themes, and rich world-building mentioned in the reviews could provoke feelings of curiosity and intellectual stimulation. The positive reactions to the movie, coupled with the praise for the director, cast, and technical aspects, suggest that viewers would be in a state of anticipation and appreciation while watching a film of this caliber."
#### Key words we could potentially work with (for this specific movie):
- overwhelmingly positive
- excited
- intrigued
- engaged
- excitement
- awe
- inspiration
- curiosity

In [19]:
import os
import pandas as pd
import openai

# Assuming you're loading environment variables where your API key is stored
# load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_sentiment(question):
    messages = [
        {"role": "system", "content": "Based on the text provided, please perform a sentiment analysis. Given that these are movie reviews, please determine which emotional state a person may be in if they were to watch a movie given the reviews. Explain why."},
        {"role": "user", "content": question}
    ]
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    
    answer = response.choices[0].message.content  # Make sure this accesses content correctly based on your error feedback
    return answer

# Load the DataFrame
movie_df = pd.read_csv('../Resources/sentiment_movies.csv')  # Adjust the path as necessary

# Limit processing to the first 10 rows of the DataFrame
movie_df_subset = movie_df.head(3)

# Apply sentiment analysis to each review in the 'cleaned_reviews' column of the subset
movie_df_subset['sentiment'] = movie_df_subset['cleaned_reviews'].apply(get_sentiment)

# Print the results for the subset
print(movie_df_subset[['cleaned_reviews', 'sentiment']].head())

# Optionally, you might want to save the results of the subset to a new CSV for review
movie_df_subset.to_csv('openai_sentiment_of_reviews_subset.csv', index=False)


                                     cleaned_reviews  \
0  dune part two surpasses even highest expectati...   
1  kung fu panda best kung fu panda film even bes...   
2  godzilla kong new empire version previous inst...   

                                           sentiment  
0  Overall, the sentiment in the reviews seems to...  
1  The sentiment analysis of the provided movie r...  
2  The sentiment analysis of the provided text in...  


C:\Users\tpete\AppData\Local\Temp\ipykernel_8516\2459655379.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df_subset['sentiment'] = movie_df_subset['cleaned_reviews'].apply(get_sentiment)


# Prompt Change
#### New Prompt:
- "Based on the text provided, please perform a sentiment analysis. Given that these are movie reviews, please determine which emotional state a person may be in if they were to watch a movie given the reviews, with up to 5 words total in your response."

- An example from Dune 2:
- "Positive sentiment; mesmerizing masterpiece with breathtaking cinematography."
- An example from Kung Fu Panda 4:
- "Excited, entertained, disappointed, bored, irritated."

In [20]:
import os
import pandas as pd
import openai

# Assuming you're loading environment variables where your API key is stored
# load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_sentiment(question):
    messages = [
        {"role": "system", "content": "Based on the text provided, please perform a sentiment analysis. Given that these are movie reviews, please determine which emotional state a person may be in if they were to watch a movie given the reviews, with up to 5 words total in your response."},
        {"role": "user", "content": question}
    ]
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    
    answer = response.choices[0].message.content  # Make sure this accesses content correctly based on your error feedback
    return answer

# Load the DataFrame
movie_df = pd.read_csv('../Resources/sentiment_movies.csv')  # Adjust the path as necessary

# Limit processing to the first 10 rows of the DataFrame
movie_df_subset = movie_df.head(3)

# Apply sentiment analysis to each review in the 'cleaned_reviews' column of the subset
movie_df_subset['sentiment'] = movie_df_subset['cleaned_reviews'].apply(get_sentiment)

# Print the results for the subset
print(movie_df_subset[['cleaned_reviews', 'sentiment']].head())

# Optionally, you might want to save the results of the subset to a new CSV for review
movie_df_subset.to_csv('adj_openai_sentiment_of_reviews_subset.csv', index=False)


                                     cleaned_reviews  \
0  dune part two surpasses even highest expectati...   
1  kung fu panda best kung fu panda film even bes...   
2  godzilla kong new empire version previous inst...   

                                           sentiment  
0  Positive sentiment; mesmerizing masterpiece wi...  
1  Excited, entertained, disappointed, bored, irr...  
2  Excitement, awe, entertaining, visually specta...  


C:\Users\tpete\AppData\Local\Temp\ipykernel_8516\2100344917.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df_subset['sentiment'] = movie_df_subset['cleaned_reviews'].apply(get_sentiment)


# Prompt 3: "Comfort levels of movies based on overview"
- Working on the **OVERVIEWS** column (**clean_overview**)
- Focusing here on 3 very broad feelings: Uncomfortable, Comfortable, or neither
- "Neither" may need to be either removed from the choices, or figure out a category of movies that would fall under (genre OR emotion-based)
- Prompt:
- "The provided text is the overview of a movie. I need you to tell me whether a watcher of this movie would be uncomfortable, comfortable, or neither."
---
Output:
- For Dune 2's overview:
    - "A watcher of this movie may feel both uncomfortable and intrigued. The plot contains elements of revenge, family destruction, difficult choices, and the prevention of a terrible future. These themes can create a sense of tension and uncertainty, which may make some viewers uncomfortable. However, the mythical journey and epic battle between good and evil can also be captivating and keep viewers engaged."
- For Kung fu 4:
    - "A watcher of this movie would likely be comfortable while watching it.
- For Kong:
    - "A watcher of this movie might feel uncomfortable due to the tense and high-stakes nature of the plot, as well as the mention of an undiscovered threat that poses a challenge to the existence of the world."


In [21]:
import os
import pandas as pd
import openai

# Assuming you're loading environment variables where your API key is stored
# load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_sentiment(question):
    messages = [
        {"role": "system", "content": "The provided text is the overview of a movie. I need you to tell me whether a watcher of this movie would be uncomfortable, comfortable, or neither."},
        {"role": "user", "content": question}
    ]
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    
    answer = response.choices[0].message.content  # Make sure this accesses content correctly based on your error feedback
    return answer

# Load the DataFrame
movie_df = pd.read_csv('../Resources/sentiment_movies.csv')  # Adjust the path as necessary

# Limit processing to the first 10 rows of the DataFrame
movie_df_subset = movie_df.head(3)

# Apply sentiment analysis to each review in the 'cleaned_reviews' column of the subset
movie_df_subset['sentiment'] = movie_df_subset['clean_overview'].apply(get_sentiment)

# Print the results for the subset
print(movie_df_subset[['clean_overview', 'sentiment']].head())

# Optionally, you might want to save the results of the subset to a new CSV for review
movie_df_subset.to_csv('overviews_openai_sentiment_of_subset.csv', index=False)


                                      clean_overview  \
0  follow mythic journey paul atreides unites cha...   
1  po gearing become spiritual leader valley peac...   
2  following explosive showdown godzilla kong mus...   

                                           sentiment  
0  A watcher of this movie may feel both uncomfor...  
1  A watcher of this movie would likely be comfor...  
2  A watcher of this movie might feel uncomfortab...  


C:\Users\tpete\AppData\Local\Temp\ipykernel_8516\3774118988.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df_subset['sentiment'] = movie_df_subset['clean_overview'].apply(get_sentiment)


# Good or Bad Movie

In [22]:
import os
import pandas as pd
import openai

# Assuming you're loading environment variables where your API key is stored
# load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_sentiment(question):
    messages = [
        {"role": "system", "content": "Determine if the provided text would be indicative of a good or bad movie."},
        {"role": "user", "content": question}
    ]
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    
    answer = response.choices[0].message.content  # Make sure this accesses content correctly based on your error feedback
    return answer

# Load the DataFrame
movie_df = pd.read_csv('../Resources/sentiment_movies.csv')  # Adjust the path as necessary

# Limit processing to the first 10 rows of the DataFrame
movie_df_subset = movie_df.head(3)

# Apply sentiment analysis to each review in the 'cleaned_reviews' column of the subset
movie_df_subset['sentiment'] = movie_df_subset['cleaned_reviews'].apply(get_sentiment)

# Print the results for the subset
print(movie_df_subset[['cleaned_reviews', 'sentiment']].head())

# Optionally, you might want to save the results of the subset to a new CSV for review
movie_df_subset.to_csv('good_bad_overview_openai_sentiment_of_subset.csv', index=False)

                                     cleaned_reviews  \
0  dune part two surpasses even highest expectati...   
1  kung fu panda best kung fu panda film even bes...   
2  godzilla kong new empire version previous inst...   

                                           sentiment  
0  This text is discussing the movie "Dune Part T...  
1  This text is indicative of a bad movie. The re...  
2  This text would be indicative of a bad movie. ...  


C:\Users\tpete\AppData\Local\Temp\ipykernel_8516\1082308815.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df_subset['sentiment'] = movie_df_subset['cleaned_reviews'].apply(get_sentiment)


In [ ]:
# 

In [ ]:
import os
import pandas as pd
import openai

# Assuming you're loading environment variables where your API key is stored
# load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_sentiment(question):
    messages = [
        {"role": "system", "content": "Determine if the provided text would be indicative of a good or bad movie."},
        {"role": "user", "content": question}
    ]
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    
    answer = response.choices[0].message.content  # Make sure this accesses content correctly based on your error feedback
    return answer

# Load the DataFrame
movie_df = pd.read_csv('../Resources/sentiment_movies.csv')  # Adjust the path as necessary

# Limit processing to the first 10 rows of the DataFrame
movie_df_subset = movie_df.head(3)

# Apply sentiment analysis to each review in the 'cleaned_reviews' column of the subset
movie_df_subset['sentiment'] = movie_df_subset['cleaned_reviews'].apply(get_sentiment)

# Print the results for the subset
print(movie_df_subset[['cleaned_reviews', 'sentiment']].head())

# Optionally, you might want to save the results of the subset to a new CSV for review
movie_df_subset.to_csv('overview_openai_sentiment_of_subset.csv', index=False)

In [45]:
movie_df_subset.head()

,Unnamed: 0,movie_id,popularity,title,vote_average,vote_count,cleaned_reviews,type_Action,type_Adventure,type_Animation,...,review_pos,review_compound,clean_overview,overview_neg,overview_neu,overview_pos,overview_compound,release_date,release_year,sentiment
6559,6559,81470,1.403,The Guy From Harlem,3.0,17,film bad hard watch must watch rifftrax track ...,True,False,False,...,0.184,-0.2500,tough streetwise private investigator al conno...,0.270,0.670,0.060,-0.8442,1977-09-01,1977,The text is negative. This is indicated by the...
6560,6560,315379,1.402,Paper Man,6.3,10,killing machine credit card mistakenly issued ...,False,False,False,...,0.170,0.7096,prank starts group college students creating f...,0.195,0.588,0.217,-0.1280,1971-11-12,1971,The text is neutral. It provides a summary of ...


In [46]:
movie_df_subset['sentiment']


6559    The text is negative. This is indicated by the...
6560    The text is neutral. It provides a summary of ...
Name: sentiment, dtype: object

In [ ]:
# def strip_sentiment(answer):
#     if "happy" in answer:
#         return "happy"
#     if "sad" in answer:
#         return "sad"
#     if "neutral" in answer:
#         return "neutral"
    
# strip_sentiment(answer)